In [1]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cmocean
import spatialpandas.dask 
import dask as da
import dask.dataframe as dd
import matplotlib.pyplot as plt
import pandas as pd

import uxarray as ux

import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import rasterize
import datashader as ds
import datashader.transfer_functions as tf
import geoviews.feature as gf

import spatialpandas as sp

import multiprocessing

from multiprocessing import Pool

In [2]:
import sys
%load_ext autoreload
%autoreload 2

sys.path.insert(1, '/glade/u/home/philipc/geocat-internal-testing/polyplot/')

from polyplot import poly_plot as pp

# Data

In [3]:
ds_base_path = "/glade/p/cisl/vast/vapor/data/Source/UGRID/NOAA-geoflow/large/"
ds_grid = ux.open_dataset(ds_base_path + "grid.nc")
ds_v1 = xr.open_dataset(ds_base_path + "v1.000000.nc")

Loading initial grid from file:  /glade/p/cisl/vast/vapor/data/Source/UGRID/NOAA-geoflow/large/grid.nc


In [4]:
ds_grid.ds

<xarray.Dataset>
Dimensions:          (nMeshFaces: 3932160, nFaceNodes: 4, nMeshNodes: 6144000, meshLayers: 20)
Coordinates:
    mesh_node_x      (nMeshNodes) float64 ...
    mesh_node_y      (nMeshNodes) float64 ...
Dimensions without coordinates: nMeshFaces, nFaceNodes, nMeshNodes, meshLayers
Data variables:
    mesh             int32 ...
    mesh_face_nodes  (nMeshFaces, nFaceNodes) uint32 ...
    mesh_depth       (meshLayers, nMeshNodes) float64 ...

In [5]:
lon_cell = ds_grid.ds['mesh_node_x'].values
lat_cell = ds_grid.ds['mesh_node_y'].values
face_nodes = ds_grid.ds['mesh_face_nodes'].values.astype(int)

In [6]:
var_dict = ds_grid.ds_var_names
ugrid_dict = dict((k, v) for k,v in var_dict.items())

In [7]:
# Specify Time and Elevation Slice
t, level = 0, 0

# Data Variable to Plot
v1 = ds_v1['v1'][t][level].values

# PolygonArray Bottleneck

In [15]:
def test_to_list(n):
    poly_array = np.ones((n, 1, 8))
    a = poly_array.tolist()

In [16]:
def test_polygon_array(n):
    poly_array = np.ones((n, 1, 8))
    polygons = sp.geometry.PolygonArray(poly_array.tolist())

In [29]:
n_list = [10**n for n in range(3, 8)]
n_list

[1000, 10000, 100000, 1000000, 10000000]

In [30]:
print("Test: tolist()")
for n in n_list:
    print("\n")
    print("Number of Polygons: {}".format(n))
    %timeit -r 3 -n 5 test_to_list(n) 

Test: tolist()


Number of Polygons: 1000
264 µs ± 72.4 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 10000
2.18 ms ± 101 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 100000
37.4 ms ± 457 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 1000000
446 ms ± 515 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 10000000
4.55 s ± 30.6 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [31]:
print("Test: PolygonArray")
for n in n_list:
    print("\n")
    print("Number of Polygons: {}".format(n))
    %timeit -r 3 -n 5 test_polygon_array(n) 

Test: PolygonArray


Number of Polygons: 1000
1.81 ms ± 115 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 10000
15.9 ms ± 204 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 100000
181 ms ± 2.06 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 1000000
1.95 s ± 8.81 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


Number of Polygons: 10000000
19.7 s ± 86.9 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
